<h1><center>Простые векторные модели текста</center></h1>

<img src="pipeline_vec.png" alt="pipeline.png" style="width: 400px;"/>

### Задача: классификация твитов по тональности

В этом занятии мы познакомимся с распространенной задачей в анализе текстов: с классификацией текстов на классы.

В рассмотренном тут примере классов будет два: положительный и отрицательный, такую постановку этой задачи обычно называют классификацией по тональности или sentiment analysis.

Классификацию по тональности используют, например, в рекомендательных системах и при анализе отзывов клиентов, чтобы понять, понравилось ли людям кафе, кино, etc.

Более подробно мы рассмотрим данную задачу и познакомимся с более сложными методами её решения в семинаре 3, а здесь разберем простые подходы, основанные на методе мешка слов.

У нас есть [данные постов в твиттере](http://study.mokoron.com/), про из которых каждый указано, как он эмоционально окрашен: положительно или отрицательно. 

**Задача**: построить модель, которая по тексту поста предсказывает его эмоциональную окраску.


Скачиваем данные: [положительные](https://drive.google.com/file/d/1mW_fUtYmRF19AXVySU0gJOIgx0-1EFgD/view?usp=sharing), [отрицательные](https://drive.google.com/file/d/1ZnsFuf-yfO3UEHlIpk7TTqfKkEMdm1EQ/view?usp=sharing).

In [3]:
# если у вас линукс / мак / collab или ещё какая-то среда, в которой работает wget, можно так:
# !wget --no-check-certificate 'https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0' -O positive.csv
# !wget --no-check-certificate 'https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv?dl=0' -O negative.csv

--2022-05-11 22:14:42--  https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0
Распознаётся www.dropbox.com (www.dropbox.com)… 162.125.70.18
Подключение к www.dropbox.com (www.dropbox.com)|162.125.70.18|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 301 Moved Permanently
Адрес: /s/raw/fnpq3z4bcnoktiv/positive.csv [переход]
--2022-05-11 22:14:42--  https://www.dropbox.com/s/raw/fnpq3z4bcnoktiv/positive.csv
Повторное использование соединения с www.dropbox.com:443.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: https://uc80ff7d37686dceff363a73ef02.dl.dropboxusercontent.com/cd/0/inline/BlEuuDZBt0E9rmKBHZcqooOIQScYnpNltLGQBoHD83VI9BO4ICLLVxUatcu0Fly232T4igosYZnQng88bdAWguWzJmlUp1q1gbQx9LD6nCc712GwAPQuO-RDBFXBrtiUgqs4kLUSNfcAeW2DFV7N8tb5TRGDkm7yDsCfiXPVubiS0A/file# [переход]
--2022-05-11 22:14:43--  https://uc80ff7d37686dceff363a73ef02.dl.dropboxusercontent.com/cd/0/inline/BlEuuDZBt0E9rmKBHZcqooOIQScYnpNltLGQBoHD83VI9BO4ICLLVxUatcu0Fly232T4igosYZnQng8

In [4]:
import pandas as pd
import numpy as np
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 800)

In [7]:
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)
df = pd.concat([positive, negative])

In [8]:
df.sample(5)

,text,label
85940,"Ресторан Прага, наконец то ужин!:) http://t.co/s75V746Uhv",positive
30750,Культурный шок: на самом попсовом радио города внезапно играет Placebo. :),positive
56905,"RT @VRSoloviev: Ни Фетисова,ни Роднину не пригласили на Олимпиаду( После скандала в прессе Фетисов получил аккредитацию.\nКто за это отвечае…",negative
24835,"@Star69Struk с почками же? У меня тож проблемы, оочень сильно боюсь оказаться в больнице ещё раз(",negative
83892,RT @Vlada188: @leonard_9901 ахуэть типирь (((0(0(,negative


Воспользуемся функцией для предобработки текста, которую мы написали в прошлом семинаре:

In [9]:
import re
from pymorphy2 import MorphAnalyzer
from functools import lru_cache
from nltk.corpus import stopwords

m = MorphAnalyzer()
regex = re.compile("[А-Яа-яA-z]+")

def words_only(text, regex=regex):
    try:
        return regex.findall(text.lower())
    except:
        return []

In [10]:
@lru_cache(maxsize=128)
def lemmatize_word(token, pymorphy=m):
    return pymorphy.parse(token)[0].normal_form

def lemmatize_text(text):
    return [lemmatize_word(w) for w in text]


mystopwords = stopwords.words('russian') 
def remove_stopwords(lemmas, stopwords = mystopwords):
    return [w for w in lemmas if not w in stopwords and len(w) > 3]

def clean_text(text):
    tokens = words_only(text)
    lemmas = lemmatize_text(tokens)
    
    return ' '.join(remove_stopwords(lemmas))

In [12]:
# from multiprocessing import Pool
from tqdm import tqdm

# with Pool(4) as p:
lemmas = list(tqdm(map(clean_text, df['text']), total=len(df)))
    
df['lemmas'] = lemmas
df.sample(5)

100%|██████████████████████████████████| 226834/226834 [05:26<00:00, 695.32it/s]


,text,label,lemmas
76226,Парни - это такие бесчувственные существа которые не понимают к сожалению эмоции других очень близких им людей. :-( Обидно!.....,negative,парень бесчувственный существо который понимать сожаление эмоция очень близкий человек обидно
15433,"@malova_o Поздравляю! А категорию потвердили? Я более-менее,погода очень плохая,сейчас+7,100% влажность и туман! Поэтому и голова болит!((",negative,malova_o поздравлять категория потвердить менее погода очень плохой влажность туман поэтому голова болеть
58134,В нашей нормальности я перестала сомневаться уже давно))))) http://t.co/jXUVJoFo8g,positive,нормальность перестать сомневаться давно http jxuvjofo
63012,"Хотела сделать коллаж дочери,херня какая-то получилась.\nЯ открыточку пришлю кароч и всё о_О",negative,хотеть сделать коллаж дочь херня получиться открыточка прислать кароч
22241,"Мама спросила, что я хочу в подарок на НГ.Я в ступоре. У меня всё есть.А то, чего нет, мне никто не может дать( #жизньболь #нг #подарки",negative,мама спросить хотеть подарок ступор никто мочь дать жизньболь подарок


Разбиваем на train и test:

In [13]:
x_train, x_test, y_train, y_test = train_test_split(df.lemmas, df.label)

## Мешок слов (Bag of Words, BoW)


In [14]:
from sklearn.linear_model import LogisticRegression 
from sklearn.feature_extraction.text import CountVectorizer

... Но сперва пару слов об n-граммах. Что такое n-граммы:

In [15]:
from nltk import ngrams

In [16]:
sent = 'Если б мне платили каждый раз'.split()
list(ngrams(sent, 1)) # униграммы

[('Если',), ('б',), ('мне',), ('платили',), ('каждый',), ('раз',)]

In [17]:
list(ngrams(sent, 2)) # биграммы

[('Если', 'б'),
 ('б', 'мне'),
 ('мне', 'платили'),
 ('платили', 'каждый'),
 ('каждый', 'раз')]

In [18]:
list(ngrams(sent, 3)) # триграммы

[('Если', 'б', 'мне'),
 ('б', 'мне', 'платили'),
 ('мне', 'платили', 'каждый'),
 ('платили', 'каждый', 'раз')]

In [19]:
list(ngrams(sent, 5)) # ... пентаграммы?

[('Если', 'б', 'мне', 'платили', 'каждый'),
 ('б', 'мне', 'платили', 'каждый', 'раз')]

Итак, мы хотим преобразовать наши обработанные данные в вектора с помощью мешка слов. Мешок слов можно строить как для отдельных слов (лемм в нашем случае), так и для n-грамм, и это может улучшать качество. 

Объект `CountVectorizer` делает простую вещь:
* строит для каждого документа (каждой пришедшей ему строки) вектор размерности `n`, где `n` -- количество слов или n-грам во всём корпусе
* заполняет каждый i-тый элемент количеством вхождений слова в данный документ

In [20]:
vec = CountVectorizer(ngram_range=(1, 1)) # строим BoW для слов
bow = vec.fit_transform(x_train) 

ngram_range отвечает за то, какие n-граммы мы используем в качестве признаков:<br/>
ngram_range=(1, 1) -- униграммы<br/>
ngram_range=(3, 3) -- триграммы<br/>
ngram_range=(1, 3) -- униграммы, биграммы и триграммы.

В vec.vocabulary_ лежит словарь: соответствие слов и их индексов в словаре:

In [21]:
list(vec.vocabulary_.items())[:10]

[('atmosphaere', 8681),
 ('интересно', 115071),
 ('представлять', 142234),
 ('https', 31398),
 ('bwzxwin', 12850),
 ('знать', 113767),
 ('измениться', 114493),
 ('nastya', 54758),
 ('простить', 144269),
 ('утро', 160527)]

In [22]:
bow[0]

<1x168887 sparse matrix of type '<class 'numpy.int64'>'
	with 3 stored elements in Compressed Sparse Row format>

Теперь у нас есть вектора, на которых можно обучать модели! 

In [23]:
clf = LogisticRegression(random_state=42, max_iter=500)
clf.fit(bow, y_train)

LogisticRegression(max_iter=500, random_state=42)

Посмотрим на качество классификации на тестовой выборке. Для этого выведем classification_report из модуля [sklearn.metrics](https://scikit-learn.org/stable/modules/classes.html#sklearn-metrics-metrics)

В качестве целевой метрики качества будем рассматривать macro average f1-score.

In [24]:
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.74      0.73      0.74     28591
    positive       0.73      0.74      0.74     28118

    accuracy                           0.74     56709
   macro avg       0.74      0.74      0.74     56709
weighted avg       0.74      0.74      0.74     56709



Попробуем сделать то же самое для триграмм:

In [25]:
vec = CountVectorizer(ngram_range=(3, 3))
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42, max_iter = 300)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.97      0.53      0.69     51454
    positive       0.16      0.85      0.27      5255

    accuracy                           0.56     56709
   macro avg       0.56      0.69      0.48     56709
weighted avg       0.90      0.56      0.65     56709



Видим, что качество существенно хуже. Ниже мы поймем, почему это так.

## TF-IDF векторизация

`TfidfVectorizer` делает то же, что и `CountVectorizer`, но в качестве значений – tf-idf каждого слова.

Как считается tf-idf:

TF (term frequency) – относительная частотность слова в документе:
$$ TF(t,d) = \frac{n_t}{\sum_k n_k} $$

`t` -- слово (term), `d` -- документ, $n_t$ -- количество вхождений слова, $n_k$ -- количество вхождений остальных слов

IDF (inverse document frequency) – обратная частота документов, в которых есть это слово:
$$ IDF(t, D) = \mbox{log} \frac{|D|}{|{d : t \in d}|} $$

`t` -- слово (term), `D` -- коллекция документов

Перемножаем их:
$$TF–IDF(t,d,D) = TF(t,d) \times IDF(i, D)$$

Ключевая идея этого подхода – если слово часто встречается в одном документе, но в целом по корпусу встречается в небольшом 
количестве документов, у него высокий TF-IDF.

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [27]:
vec = TfidfVectorizer(ngram_range=(1, 1))
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42, max_iter = 500)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.70      0.74      0.72     26386
    positive       0.76      0.72      0.74     30323

    accuracy                           0.73     56709
   macro avg       0.73      0.73      0.73     56709
weighted avg       0.73      0.73      0.73     56709



В этот раз получилось хуже, чем с помощью простого CountVectorizer, то есть использование tf-idf не дало улучшений в качестве. 

## О важности эксплоративного анализа

Иногда в ходе стандартного препроцессинга теряются важные признаки. Посмотрим, что будет если не убирать пунктуацию?

In [28]:
df.sample()

,text,label,lemmas
2903,У меня за стеной у кого-то веселье. Пьяные мужские голоса поют что-то в руссконародном духе. Смешно)),positive,стена веселие пьяный мужской голос петь руссконародный смешно


In [29]:
df['new_lemmas'] = df.text.apply(lambda x: x.lower())
df.sample(3)

,text,label,lemmas,new_lemmas
16330,RT @qohavaboweka: удалить....нужно срочно удалить человек 500 из ЖЖ... как это сделать никого не обидев(((,negative,qohavaboweka удалить нужно срочно удалить человек сделать никто обидеть,rt @qohavaboweka: удалить....нужно срочно удалить человек 500 из жж... как это сделать никого не обидев(((
83975,"@nadi_yurkovets столько приятностей в один день мне еще не говорили, бро,я с гордостью скажу, что ты первая! и знай, что я тебя тоже люблю:)",positive,nadi_yurkovets столько приятность день говорить гордость сказать первый знать любить,"@nadi_yurkovets столько приятностей в один день мне еще не говорили, бро,я с гордостью скажу, что ты первая! и знай, что я тебя тоже люблю:)"
67219,я какая-то не такая:( изменилась может быть...,negative,измениться мочь,я какая-то не такая:( изменилась может быть...


In [30]:
x_train, x_test, y_train, y_test = train_test_split(df.new_lemmas, df.label)

In [31]:
from nltk import word_tokenize

vec = TfidfVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42, max_iter = 300)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       1.00      1.00      1.00     27859
    positive       1.00      1.00      1.00     28850

    accuracy                           1.00     56709
   macro avg       1.00      1.00      1.00     56709
weighted avg       1.00      1.00      1.00     56709



Как можно видеть, если оставить пунктуацию, то все метрики равны 1. 

In [32]:
len(vec.vocabulary_), len(clf.coef_[0])

(260034, 260034)

In [37]:
importances = list(zip(vec.vocabulary_, clf.coef_[0]))
importances[0]

('вчера', 0.2136572091594278)

In [34]:
sorted_importances = sorted(importances, key = lambda x: -x[1])
sorted_importances[:10]

[('весело', 58.44800689285947),
 ('maxnest', 26.87589763915838),
 ('metdasha', 10.474993882107558),
 ('mooney_lupin', 9.174847685759397),
 ('вкусне', 7.901848649134337),
 ('шмотки', 7.3767992135484715),
 ('ходить', 7.017478527197771),
 ('bagusbdman', 6.136608669599598),
 ('уставшая', 4.840771075371882),
 ('жизни.спасибо', 3.066545371925849)]

Посмотрим, как один из наиболее значительных токенов справится с классификацией безо всякого машинного обучения:

In [38]:
cool_token = ')'
pred = ['positive' if cool_token in tweet else 'negative' for tweet in x_test]
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       1.00      0.85      0.92     32878
    positive       0.83      1.00      0.91     23831

    accuracy                           0.91     56709
   macro avg       0.91      0.92      0.91     56709
weighted avg       0.93      0.91      0.91     56709



Можно видеть, что это уже позволяет достаточно хорошо классифицировать тексты.

## Символьные n-граммы

Теперь в качестве признаком используем, например, униграммы символов:

In [40]:
vec = CountVectorizer(analyzer='char', ngram_range=(1, 1))
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42, max_iter=1000)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.99      1.00      1.00     27799
    positive       1.00      0.99      1.00     28910

    accuracy                           1.00     56709
   macro avg       1.00      1.00      1.00     56709
weighted avg       1.00      1.00      1.00     56709



Таким образом, становится понятно, почему на этих данных качество классификации 1. Так или иначе, на символах классифицировать тоже можно.

Ещё одна замечательная особенность символьных признаков: токенизация и лемматизация не нужна, можно использовать такой подход для языков, у которых нет готовых анализаторов.

## Итоги

 На этом занятии мы
* познакомились с задачей бинарной классификации текстов.

* научились строить простые признаки на основе метода "мешка слов" с помощью библиотеки sklearn: CountVectorizer и TfidfVectorizer.

* использовали для классификации линейную модель логистической регрессии.

* поняли, что многое зависит от подхода к предобработки текста и от признаков, которые используются в модели.

* увидели, что в некоторых задачах важно использование каждого символа из текста, в том числе пунктуации.

На следующих занятиях мы рассмотрим более сложные модели построения признаков и классификации текстов.